# GitHub Issues Summarizer Webex Chatbot using OpenAI GPT API

This project aims to create a chatbot in Webex that summarizes GitHub issues using the ChatGPT API and sends a daily summary to a team manager.

## Prerequisites:
1. GitHub API Token
2. OpenAI API Key
3. Webex Bot Token
4. Python libraries: `requests`, `schedule`, `openai`

### Install Required Libraries
```python
!pip install requests schedule openai
```

---

## 1. Fetch New Issues from GitHub
We will start by retrieving the latest GitHub issues from a repository using the GitHub API. Only new issues (within the last 24 hours) will be fetched.


In [ ]:
import requests
from datetime import datetime, timedelta

# GitHub API Token and repository info
GITHUB_TOKEN = 'your_github_token'
REPO_OWNER = 'repo_owner'  # e.g., 'octocat'
REPO_NAME = 'repo_name'  # e.g., 'Hello-World'

# Function to get new issues from the GitHub repository
def get_new_issues(since_time):
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/issues"
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    params = {
        "since": since_time.isoformat(),  # Get issues created since this time
        "state": "open"  # Only fetch open issues
    }

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # List of issues
    else:
        print("Error fetching issues from GitHub:", response.status_code)
        return []

# Example usage: Fetch issues created in the last 24 hours
last_day = datetime.utcnow() - timedelta(days=1)
new_issues = get_new_issues(last_day)

---

## 2. Summarize GitHub Issues Using OpenAI GPT API

### Why Use ChatGPT API?
For summarizing the GitHub issues, the **ChatGPT API** (from OpenAI) is ideal due to the following reasons:

- **Text Summarization Capabilities:** ChatGPT models are designed to handle complex text data, making them well-suited for summarizing technical issues into concise, human-readable summaries.
- **Natural Language Understanding:** ChatGPT can process unstructured data, transforming raw issue details from GitHub into a coherent and meaningful report.
- **Customization of Prompts:** You can guide the model using specific instructions, ensuring the summaries are tailored to the manager's needs.

### API Endpoint Used: /v1/chat/completions
The **/v1/chat/completions** endpoint of OpenAI's API is used for generating conversational responses. In this case, it helps create a summary of the GitHub issues.

#### Model Selection:
- **GPT-3.5:** Faster and cost-efficient, suitable for simpler tasks.
- **GPT-4:** More accurate, offering better performance on complex issues.

You can craft prompts that direct ChatGPT to provide concise summaries of GitHub issues, making it easy for managers to understand critical points.

Now, let's see the code that uses the ChatGPT API to summarize the issues.


In [ ]:
import openai

# OpenAI API Key
OPENAI_API_KEY = 'your_openai_api_key'

openai.api_key = OPENAI_API_KEY

# Function to summarize GitHub issues using ChatGPT
def summarize_issues(issues):
    if not issues:
        return "No new issues to report."

    # Prepare a prompt with issue details
    issue_texts = [f"Issue #{issue['number']}: {issue['title']}
Description: {issue['body']}
" for issue in issues]
    prompt = "Summarize the following GitHub issues in a concise report:
" + "
".join(issue_texts)
    
    # Call ChatGPT API
    response = openai.ChatCompletion.create(
        model="gpt-4",  # You can also use gpt-3.5-turbo for cost efficiency
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes GitHub issues for daily reporting."},
            {"role": "user", "content": prompt},
        ]
    )

    # Extract the summary from the response
    summary = response['choices'][0]['message']['content']
    return summary

# Example usage: Summarize the new issues
summary = summarize_issues(new_issues)
print(summary)

---

---

## 3. Send the Summary to Webex Using Webex API

### 3.1. Create a Webex Bot and Get the Bot Token:

To create a bot on Webex and obtain a token, follow these steps:

1. **Sign Up for Webex:**
   - Go to the [Webex Developer Portal](https://developer.webex.com/) and sign in with your Webex account. If you don't have an account, you can create one for free.

2. **Create a New Bot:**
   - Once signed in, navigate to **My Apps** in the top-right corner.
   - Click on **Create a New App** and select **Create a Bot**.
   - Fill in the bot's details (e.g., bot name, username, icon). These details will be visible to the users when they interact with your bot.
   - After creating the bot, you'll be provided with a **Bot Access Token**. **Copy and save this token**, as you’ll need it to authenticate your bot’s API requests.

3. **Bot Access Token Example:**
   - The token will look something like this:
     ```
     Z2F.....YXJk.TEF....-zo
     ```

### 3.2. Get the Webex Room ID:

The **Room ID** is required to send messages to a particular space (room) in Webex. You can retrieve this by making an API call to list the rooms or spaces where the bot is a member.

1. **Add the Bot to a Webex Room:**
   - Before you can send messages, the bot must be added to the space (room) where you want it to post messages. You can either add the bot manually to a Webex space or create a space via the Webex Developer API.
   
2. **Retrieve the Room ID Using the Webex API:**
   - Use the `/v1/rooms` endpoint of the Webex API to retrieve the Room ID where the bot is a member.
   - Here's an example of how to get the Room ID using Python
   - This script will print out the **Room Name** and **Room ID** for each space where the bot is a member.
   - Choose the appropriate Room ID for your bot to post messages.

In [ ]:
   import requests

   # Your Webex Bot Token
   WEBEX_BOT_TOKEN = 'your_webex_bot_token'

   def get_rooms():
       url = "https://webexapis.com/v1/rooms"
       headers = {
           "Authorization": f"Bearer {WEBEX_BOT_TOKEN}",
           "Content-Type": "application/json"
       }

       response = requests.get(url, headers=headers)
       if response.status_code == 200:
           rooms = response.json()['items']
           for room in rooms:
               print(f"Room Name: {room['title']}, Room ID: {room['id']}")
       else:
           print(f"Failed to fetch rooms: {response.status_code}")

   # Get a list of rooms where the bot is a member
   get_rooms()



### 3.3. Send a Message to Webex Using the Webex API:

Once you have the **Bot Token** and the **Room ID**, you can send the summarized GitHub issues to Webex.

Here’s the full example code for sending a message:

In [ ]:
import requests

# Webex Bot Token and Room ID
WEBEX_BOT_TOKEN = 'your_webex_bot_token'
WEBEX_ROOM_ID = 'your_webex_room_id'  # Room where the bot will post messages

# Function to send message to Webex
def send_message_to_webex(summary):
    url = "https://webexapis.com/v1/messages"
    headers = {
        "Authorization": f"Bearer {WEBEX_BOT_TOKEN}",
        "Content-Type": "application/json"
    }
    data = {
        "roomId": WEBEX_ROOM_ID,
        "text": summary
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        print("Message sent to Webex successfully.")
    else:
        print("Error sending message to Webex:", response.status_code)

# Example usage: Send the summary to Webex
send_message_to_webex(summary)

---

## 4. Schedule the Script to Run Daily at 9:00 AM
We will use the `schedule` library to run the entire process daily at a specific time.


In [ ]:
import schedule
import time

# Function to run the entire process
def run_daily_summary():
    # 1. Fetch new GitHub issues
    last_day = datetime.utcnow() - timedelta(days=1)
    new_issues = get_new_issues(last_day)

    # 2. Summarize the issues
    summary = summarize_issues(new_issues)

    # 3. Send the summary to Webex
    send_message_to_webex(summary)

# Schedule the job to run every day at 9:00 AM
schedule.every().day.at("09:00").do(run_daily_summary)

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(60)

---

## Sample Data for Testing:
Here’s an example of a simple GitHub issue for testing purposes:

```json
[
    {
        "number": 101,
        "title": "Bug in the user authentication module",
        "body": "The user authentication fails when a new user is created.",
        "created_at": "2023-09-25T10:00:00Z"
    },
    {
        "number": 102,
        "title": "Add feature for exporting reports",
        "body": "Users should be able to export their reports in PDF and CSV formats.",
        "created_at": "2023-09-25T12:00:00Z"
    }
]
```